In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.io import imread,imsave,imshow
from skimage.color import rgb2gray
from scipy.stats import norm, kurtosis, entropy, skew
from skimage.feature import graycoprops,graycomatrix
import cv2
from glob import glob
import os
import csv
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier as mlp
from sklearn.metrics import accuracy_score as acc
import math
import random

In [10]:
def geraNum(n, tipo='i'):
    resultado = []
    for i in range(n):
        if tipo == 'i':
            resultado.append(random.randint(0, 100))
        else:
            resultado.append(round(random.uniform(0, 1), 3))
    return resultado

def geraValoresRandom():
    k = geraNum(4)
    e = geraNum(4)
    c = geraNum(4)
    gama = geraNum(4, 'f')
    return k, e, c, gama


In [11]:
k, e, c, gama = geraValoresRandom()

### Alongamento:

Fórmula:
    $$ s = T(r) = \frac{1}{1 + \frac{k}{r}^{E} }  $$
<ul>
    <li> k é a constante de alongamento</li>
    <li> E é o expoente de alongamento</li>
    <li> r é o pixel calculado em questão</li>
</ul>

In [ ]:
def alongamento(k, r, e):
    return 1 / ( 1 + ( ( k / r ) ** e ) )

### Negativo:

Fórmula:
    $$s = L - 1 - r$$

In [ ]:
def negativo(r, l=255):
    return l - 1 - r

### Logarítmico:

Fórmula:
    $$s = c\log(1+r)$$

In [ ]:
def logaritmico(r, c):
    return c * math.log(1 + r)

### Potência:

Fórmula:
    $$s = cr^{\gamma}$$

In [ ]:
def potencia(c, r, gama):
    return c * (r ** gama)